# Symbology conversion using Datascope Select API

In [1]:
import csv

In [2]:
uName = '-- DSS USERNAME --'
pw = '-- DSS PASSWORD --'

inputFileName = 'input.csv'
outputFileName = 'output.csv'

In [3]:
# execute the helper functions defined for accessing DSS REST API calls 
%run DSSDefines.ipynb

In [4]:
# Read the input CSV file
# Format: (Identifier Type, Identifier)

inpList = []
with open(inputFileName) as csvfile:
    rdr = csv.reader(csvfile)
    for row in rdr:
        inpList.append(row)

In [ ]:
# Login to DSS and get a get an access token
DSS_login(uName, pw)

In [6]:
# Form the request message payload with input symbols
csvIdentifiers = []

for idn in inpList:
    iObj = {
        "IdentifierType": idn[0],
        "Identifier": idn[1]
    }
    csvIdentifiers.append(iObj)


In [7]:
# Send the symbology conversion request
fieldNames = ['RIC', 'CUSIP', 'ISIN', 'SEDOL', 'Company Name', 'Currency Code']

rBody = {
    "ExtractionRequest": {
        "@odata.type": "#DataScope.Select.Api.Extractions.ExtractionRequests.TermsAndConditionsExtractionRequest",
        "ContentFieldNames": fieldNames,
        "IdentifierList": {
            "@odata.type": "#DataScope.Select.Api.Extractions.ExtractionRequests.InstrumentIdentifierList",
            "InstrumentIdentifiers": csvIdentifiers,
            "ValidationOptions": {
                "AllowHistoricalInstruments": True,
                "AllowInactiveInstruments": True,
                "AllowOpenAccessInstruments": False
            },
            "UseUserPreferencesForValidationOptions": False
        }
    }
}

convertedJ = DSS_postAsync('/Extractions/ExtractWithNotes', rBody)

In [8]:
# Save the converted symbols into a CSV file
with open(outputFileName, 'w', newline='') as outfile:
    owriter = csv.writer(outfile)
    headerKeys = convertedJ['Contents'][0].keys()
    owriter.writerow(list(headerKeys))

    for itm in convertedJ['Contents']:
        thisrow = []
        for key in headerKeys:
            thisrow.append(itm[key])

        owriter.writerow(thisrow)
